This script creates the data base. Uses local module to populate

In [168]:
import pandas as pd
import mysql.connector

con = mysql.connector.connect(user='root', password='qqqqqq',
                               host='127.0.0.1',database='warranties')
cur = con.cursor()

Insert information into the table

In [154]:
cur.execute('''DROP TABLE customers''')

In [ ]:
db = pd.read_sql('''SELECT * FROM customers''',con)

In [114]:
cur.execute('''ALTER TABLE importers ADD identity CHAR(30)''')

In [113]:
# View the first few columns
db.head()
# This is how things are committed to the database
con.commit()

Now the same for generating the importer table, the model table. These are the master tables

In [155]:
cur.execute('''CREATE TABLE customers (
                cID INT AUTO_INCREMENT,
                first_name CHAR(30),
                last_name CHAR(30),
                mob_number CHAR(12),
                region TINYINT,
                PRIMARY KEY (cID)
                )
            ''')

In [157]:
cur.execute('''CREATE TABLE importers (
                iID INT NOT NULL AUTO_INCREMENT,
                identity CHAR(30),
                PRIMARY KEY (iID)
                )
            ''')

In [158]:
cur.execute('''CREATE TABLE product_models (
                mID INT AUTO_INCREMENT,
                model CHAR(20),
                is_verified BOOLEAN,
                PRIMARY KEY (mID)
                )
            ''')

In [159]:
cur.execute('''CREATE TABLE products (
                pID INT AUTO_INCREMENT,
                ser_num CHAR(30),
                model CHAR(20),
                PRIMARY KEY (pID)
                )
            ''')

In [160]:
cur.execute('''CREATE TABLE product_sellers (
                iID INT,
                pID INT,
                imp_date DATE,
                PRIMARY KEY (iID, pID)
                )
            ''')

In [161]:
cur.execute('''CREATE TABLE warranties (
                cId INT,
                pID INT,
                ser_num CHAR(30),
                reg_date DATE,
                exp_date DATE,
                PRIMARY KEY (cID, pID)
                )
            ''')

These are useful functions for getting a date between a certain range. 

In [ ]:
# Gets a random date between two dates
def strTimeProp(start, end, format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    
    A demonstration of how to use this function
    randomDate("1/1/2000","1/1/2001",r.random())
    
    Output is a timedate.date structure
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))

# This is a helper function for strTimeProp
def randomDate(start, end, prop):
    return strTimeProp(start, end, '%Y-%m-%d', prop)

# Adds somewhere up to a month to the start date
def addMonth(start):
    """ Assume we're in format '%Y-%m-%d', adds up to a month onto the 
    start date
    Input string format date
    Output timedate.date structure
    
    # How the function works
    date = '2010-1-1'
    newDate = addMonth(date)
    print date
    print newDate
    """
    tempTime = time.mktime(time.strptime(start,'%Y-%m-%d'))
    endTime = tempTime + 24*3600*7*4
    tempTime = time.strftime('%Y-%m-%d', time.localtime(tempTime))
    endTime = time.strftime('%Y-%m-%d', time.localtime(endTime))
    
    return randomDate(tempTime, endTime, r.random())

def getWarrantyEnd(startDate,yearsValid):
    """ Adds a number of years onto the given date. 
        Input string
        Output timedate.date structure
        
        # Demonstration
        date = '2010-1-1'
        warrantyYears = 10
        print date
        print getWarrantyEnd(date,warrantyYears)
    """
    tempTime = time.mktime(time.strptime(startDate,'%Y-%m-%d'))
    tempTime = time.strftime('%Y-%m-%d', time.localtime(tempTime +
                                            3600*24*7*52*yearsValid))
    
    return tempTime

Now all tables have been created they need to be populated

In [166]:
# Info for the Customer Table
import random as r
import time
import datetime

nProducts = 500
nImporters = 10
nCustomers = 400
nWarranties = 200
nModels = 20

nameList = ['alex','barry','cedric','dave','eric','freddy','george',
            'helen','isla','jim','kerry','larry','mona','neville',
            'ophelia','pat','roger','steve','tash','will','xena!']
surnameList = ['adams','brompton','clibbon','douglas','free','humphrey',
               'iglasia','garcia','clifford','smith','jones']
    


def getCustomer(forename_list, surname_list):
    forename = forename_list[r.randint(0, len(forename_list)-1)]
    surname = surname_list[r.randint(0, len(surname_list)-1)]
    mob_num = ['+']
    for x in range(0,9):
        mob_num.append(str(r.randint(0,9)))
    region = str(r.randint(0,15))
    return (forename, surname, "".join(mob_num), region)

# Test the output
testCustomer = getCustomer(nameList, surnameList)
print testCustomer

('cedric', 'clibbon', '+447453116', '1')


In [170]:
for i in range(0,nCustomers):
    temp = getCustomer(nameList, surnameList)
    customer = {
        'forename': temp[0],
        'surname': temp[1],
        'mob_num': temp[2],
        'region': temp[3]
        }
    cur.execute('''INSERT into customers
                (first_name, last_name, mob_number, region) 
                VALUES (%(forename)s, %(surname)s, %(mob_num)s, %(region)s)
                ''', customer)

In [171]:
df = pd.read_sql('''SELECT * FROM customers''',con)
df.head()

,cID,first_name,last_name,mob_number,region
0,1,pat,clifford,+147873016,9
1,2,dave,garcia,+206650059,3
2,3,will,clibbon,+714976821,1
3,4,freddy,garcia,+031404058,12
4,5,neville,jones,+513214502,8


In [162]:
# For Importers Table - don't need to get anything as this is only
# made up of the key for now

# Make the importers table
for i in range(0,nImporters):
    imp_ident = ''.join([r.choice('0123456789ABCDEF') for x in range(12)])
    cur.execute('''INSERT into importers (identity) values (%s)''', (imp_ident,))

In [163]:

df = pd.read_sql('''SELECT * FROM importers''',con)
df


,iID,identity
0,1,BC9D919CF2BA
1,2,EE50714230A5
2,3,DE2C26AFF611
3,4,ABD893826CAF
4,5,F7EB674877ED
5,6,584F1F8B93B0
6,7,CE1ECFB9EF56
7,8,18D5F3C947B7
8,9,D89AAAD9CF37
9,10,8336F5B90C12


In [ ]:
# cur.execute('''INSERT into importers DEFAULT VALUES''')
# This doesn't work - WHY?

In [ ]:
### For Product_Seller
# Get number of sellers
df = pd.read_sql('''SELECT * FROM importers''', con)
for i in range(0,nProducts):
    print 'a'
    

# For Product

# For Warranty

In [ ]:
cur.execute('''INSERT into customers (first_name,
                last_name, mob_number, region)
                values (%s, %s, %s, %s)''',
            ('joe','bloggs','+44123456789','0'))

In [ ]:
# For Models Table
def getProduct_model():
    if r.randint(0,1) == 0:
        is_verified = True
    else:
        is_verified = False
    model = ''.join([r.choice('0123456789ABCDEF') for x in range(6)])
    return (is_verified, model)        

In [164]:
for i in range(0,nModels):
    temp = getProduct_model()
    product_model = {
        'is_verified': temp[0],
        'model': temp[1]
    }
    cur.execute('''INSERT into product_models
                (is_verified, model) 
                VALUES (%(is_verified)s, %(model)s)
                ''', product_model)

In [165]:
df = pd.read_sql('''SELECT * FROM product_models''',con)
df.head(10)

,mID,model,is_verified
0,1,566265,0
1,2,50498F,1
2,3,8FE3A3,0
3,4,2BE3F7,1
4,5,CDEDF3,0
5,6,6490D2,1
6,7,88BE21,1
7,8,66C940,0
8,9,DE1312,0
9,10,1BA753,1


Now add in the product sellers. This is the order it should actually occur in, and the creation of a product will occur at the same time.

In [ ]:
# A function to create a product instance. 